In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from collections import Counter
from string import punctuation

# Checking IMBD sentiment analysis 

Analyse random subset of 10k examples to test the presence of which words are candidate distractors

In [2]:
attr_path = 'list_attr_celeba.txt'
with open(attr_path, 'r') as f:
    text = f.readlines()
data = []
col_names = text[1].split()
for l in text[2:]:
    row = l.split()
    row = [int(x) for x in row[1:]]
    data.append(np.array(row))
col_names = np.array(col_names)    

In [3]:
df = pd.DataFrame(data = data, columns = col_names)
df = (df + 1)/2
df.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [43]:
# spread between 
spread = np.abs(np.round(np.sum(data, axis = 0)/len(df), 2))
balance = spread < 0.2
bs = col_names[balance]
for i, j in zip(bs, spread[balance]): print (i, j)

Attractive 0.03
High_Cheekbones 0.09
Male 0.17
Mouth_Slightly_Open 0.03
Smiling 0.04
Wearing_Lipstick 0.06


In [44]:
imbalance = np.logical_and(spread > 0.40, spread < 0.6)
ubs = col_names[imbalance]
ubs

array(['Arched_Eyebrows', 'Bags_Under_Eyes', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Brown_Hair', 'Oval_Face', 'Pointy_Nose',
       'Straight_Hair', 'Young'], dtype='<U19')

In [45]:
print('both, only even, only niche, neither')
for b in bs:
    for u in ubs:
        both_vec = np.logical_and(df[b], df[u])
        both = np.abs(np.round(1.0 * np.sum(both_vec, axis = 0)/len(df), 3))
        
        only_b_vec = np.logical_and(df[b], np.logical_not(df[u]))
        only_b = np.abs(np.round(1.0 * np.sum(only_b_vec, axis = 0)/len(df), 3))
        
        only_ub_vec = np.logical_and(np.logical_not(df[b]), df[u])
        only_ub = np.abs(np.round(1.0 * np.sum(only_ub_vec, axis = 0)/len(df), 3))
        
        neither_vec = np.logical_and(np.logical_not(df[b]), np.logical_not(df[u]))
        neither = np.abs(np.round(1.0 * np.sum(neither_vec, axis = 0)/len(df), 3))
        
        vals = np.array([both, only_b, only_ub, neither])
        if np.sum(vals <= 0.15) == 1:
            rest = vals[vals > 0.15]
            i, j, k = rest
            diffs = np.abs(np.array([i-j, j-k, k-i]))
            if not np.sum(diffs > 0.21):
                print(b, u)
                print(vals)
                print('***********')


both, only even, only niche, neither
Attractive Oval_Face
[0.189 0.323 0.095 0.393]
***********
Male Arched_Eyebrows
[0.022 0.394 0.245 0.339]
***********
Male Oval_Face
[0.092 0.325 0.192 0.391]
***********
Male Pointy_Nose
[0.068 0.349 0.21  0.373]
***********
